In [159]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import openpyxl
import time
from selenium.common.exceptions import TimeoutException
import sys

#### 最終版591爬蟲 爬取台灣所有中古屋房仲電話

In [175]:
locations_591 = {
    #'新北市':"https://sale.591.com.tw/?shType=list&regionid=3&totalRows=53074&firstRow=0",
    '桃園市':"https://sale.591.com.tw/?shType=list&regionid=6&firstRow=1980&totalRows=38755",
    #'台北市':"https://sale.591.com.tw/?shType=list&regionid=1&firstRow=4230&totalRows=23795",
    #'新竹縣':"https://sale.591.com.tw/?shType=list&regionid=5&firstRow=4860&totalRows=10467",
    #'新竹市':"https://sale.591.com.tw/?shType=list&regionid=4&firstRow=2730&totalRows=7994",
    #'基隆市':"https://sale.591.com.tw/?shType=list&regionid=2&firstRow=4380&totalRows=4672",
    #'宜蘭縣':"https://sale.591.com.tw/?shType=list&regionid=21&firstRow=2520&totalRows=3567",
    #'台中市':"https://sale.591.com.tw/?shType=list&regionid=8&firstRow=840&totalRows=68279",
    #'苗栗縣':"https://sale.591.com.tw/?shType=list&regionid=7&totalRows=2492&firstRow=0",
    #'彰化縣':"https://sale.591.com.tw/?shType=list&regionid=10&firstRow=810&totalRows=2467",
    '南投縣':"https://sale.591.com.tw/?shType=list&regionid=11&totalRows=803&firstRow=0",
    '雲林縣':"https://sale.591.com.tw/?shType=list&regionid=14&totalRows=520&firstRow=0",
    '高雄市':"https://sale.591.com.tw/?shType=list&regionid=17&totalRows=24620&firstRow=0",
    '台南市':"https://sale.591.com.tw/?shType=list&regionid=15&totalRows=11058&firstRow=0",
    '屏東縣':"https://sale.591.com.tw/?shType=list&regionid=19&totalRows=1715&firstRow=0",
    '嘉義市':"https://sale.591.com.tw/?shType=list&regionid=12&totalRows=1031&firstRow=0",
    '嘉義縣':"https://sale.591.com.tw/?shType=list&regionid=13&totalRows=503&firstRow=0",
    '花蓮縣':"https://sale.591.com.tw/?shType=list&regionid=23&totalRows=2349&firstRow=0",
    '台東縣':"https://sale.591.com.tw/?shType=list&regionid=22&totalRows=128&firstRow=0",
    '金門縣':"https://sale.591.com.tw/?shType=list&regionid=25&totalRows=55&firstRow=0",
    '澎湖縣':"https://sale.591.com.tw/?shType=list&regionid=24&totalRows=29&firstRow=0",
    '連江縣':"https://sale.591.com.tw/?shType=list&regionid=26&totalRows=1&firstRow=0"
}

In [180]:
# 讀取現有的 Excel 檔案
file_path = '591房仲手機號碼.xlsx'
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

for loc,web_link in locations_591.items(): #台灣所有地區
    #重新開始新的session
    options = Options()
    #options.add_argument('--headless')  # 選擇無頭瀏覽模式
    options.add_argument("--disable-notifications") #取消網頁中的彈出視窗
    driver = webdriver.Chrome(options=options)
    print("------------------------------")
    print("爬蟲{}頁面中".format(loc))
    # 591 中古屋頁面
    older_house = web_link
    wait = WebDriverWait(driver, 30)
    outer_loop_flag = False
    driver.get(older_house)
    


    # 定義一個變數，表示最大嘗試次數
    max_attempts = 3
    attempts = 0
    consecutive_exist_count = 0
    while attempts < max_attempts:
        try:
            container = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='container']"))
            )        
            # container = driver.find_element(By.XPATH, "//*[@id='container']")
            app = container.find_element(By.XPATH, "//*[@id='app']")
            clearfix = app.find_element(By.XPATH, "//*[@class='clearfix relative']")
            main_area = clearfix.find_element(By.XPATH, "//*[@class='main-area']")
            page_elements = main_area.find_element(By.XPATH, "//*[@class='pages']")
            pageNum = page_elements.find_elements(By.XPATH, "//*[@class='pageNum-form']")
            # step 1 --------取得頁面總數量-----------
            last_page_number = pageNum[-1].text
            print("頁面總數量:",last_page_number)
            # 如果成功找到元素，則離開迴圈
            break
        except IndexError as e :
            print(f"發生 IndexError: {e} Retrying...")
            time.sleep(1)
            attempts += 1



    for page in range(int(last_page_number) - 1): # 換頁執行 直到全部執行完畢
        if outer_loop_flag == True:
            break
        # step 2 --------取得該頁總數量-----------
        # 使用 XPath 定位 <section> 元素 / 使用 XPath 定位具有特定 class 的元素
        try:
            container = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='container']"))
            )
            time.sleep(0.5)   
            app = container.find_element(By.XPATH, "//*[@id='app']")
            clearfix = app.find_element(By.XPATH, "//*[@class='clearfix relative']")
            main_area = clearfix.find_element(By.XPATH, "//*[@class='main-area']")
            houseList_body = main_area.find_element(By.XPATH, "//*[@class='houseList-body']")
            # 取得 頁面所有標題的hyperlink elements
            houseList_title = wait.until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "houseList-item-title"))
            )
        except IndexError as e :
            print(f"發生 Error: {e} ")
            continue
        # # 記住一開始頁面的窗口號碼
        main_window = driver.window_handles[0]
        # step 3 --------從第一個開始執行-----------
        for i in range(1,len(houseList_title)): #排除第一個 因為第一個推薦的有時長的不一樣
            if outer_loop_flag == True:
                break
            if i == 4 :
                continue
            try:
                container = wait.until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='container']"))
                )
                time.sleep(0.5)  
                app = container.find_element(By.XPATH, "//*[@id='app']")
                clearfix = app.find_element(By.XPATH, "//*[@class='clearfix relative']")
                main_area = clearfix.find_element(By.XPATH, "//*[@class='main-area']")
                houseList_body = main_area.find_element(By.XPATH, "//*[@class='houseList-body']")
                houseList_title = wait.until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "houseList-item-title"))
                )
                # # 點擊 進入新頁面
                # 使用滾動到可見範圍的方式點擊元素
                actions = ActionChains(driver)
                actions.move_to_element(houseList_title[i])
                #driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", houseList_title[i])
                actions.click(houseList_title[i])
                actions.perform()
                
                # 記住新窗口的句柄
                new_window_handle = None

                # 等待新窗口或標籤，然後切換到它
                try:
                    new_window_handle = wait.until(lambda driver: len(driver.window_handles) > 1)
                except TimeoutException:
                    print("TimeoutException: No new window found.")
                    continue  # 跳過這一次循環，繼續下一次

                # 更新窗口句柄
                new_window_handle = [handle for handle in driver.window_handles if handle != main_window][0]

                # 切換到新窗口
                driver.switch_to.window(new_window_handle)

                element = wait.until(EC.presence_of_element_located((By.ID , "container")))
            # 使用 XPath 定位 <section> 元素
                #elements2 = element.find_element(By.CLASS_NAME, "img-and-info clearfix")
                elements2 = element.find_element(By.XPATH, "//section[@class='img-and-info clearfix']")
                # 取得元素的 outerHTML
                #element2_html = elements2.get_attribute("outerHTML")
                elements3 = elements2.find_element(By.CLASS_NAME, "info-box")
                elements4 = elements3.find_element(By.CLASS_NAME, "info-box-host")
                elements5 = elements4.find_element(By.CLASS_NAME, "info-host-two ")
                elements6 = elements5.find_element(By.CLASS_NAME, "info-host-word")
                phone_num = elements6.text
                phone_num = phone_num.strip()
                if not any(cell.value == phone_num for cell in sheet['A']) and phone_num: #hone_num not in df["accs_num"].values:
                    # 寫入新資料
                    last_row = sheet.max_row + 1
                    sheet.cell(row=last_row, column=1, value=phone_num)
                    sheet.cell(row=last_row, column=2, value=loc)
                    # 儲存檔案
                    workbook.save(file_path)
                    print('手機號碼 {} 寫入df'.format(phone_num))
                    # 重置計數器
                    consecutive_exist_count = 0

                    # 關閉原始窗口
                    driver.close()

                    # 切換回主窗口
                    driver.switch_to.window(main_window)
                else:
                    print('手機號碼 {} 已存在於df中'.format(phone_num))
                    consecutive_exist_count += 1

                    # 關閉原始窗口
                    driver.close()

                    # 切換回主窗口
                    driver.switch_to.window(main_window)

                    # 如果連續存在十次，中斷迴圈
                    if consecutive_exist_count >= 61:
                        print('手機號碼 {} 連續61次存在於df中，中断迴圈'.format(phone_num))
                        outer_loop_flag = True
            except Exception as e:
                # 捕獲例外並處理
                print(f"An error occurred: {e}")
                continue
                #driver.get(older_house)
        # step 4 --------切換下一頁-----------
        # 點擊下一頁
        # 使用 XPath 定位 <section> 元素 / 使用 XPath 定位具有特定 class 的元素

        try:
            container = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='container']"))
            )

            app = container.find_element(By.ID, "app")   
            #app = container.find_element(By.XPATH, "//*[@id='app']")
            clearfix = app.find_element(By.XPATH, "//*[@class='clearfix relative']")
            main_area = clearfix.find_element(By.XPATH, "//*[@class='main-area']")
            pages = main_area.find_element(By.XPATH, "//*[@class='pages']")
            #pageNext = pages.find_element(By.XPATH, "//*[@class='pageNext']")
            #pageNext = pages.find_element(By.XPATH, "//a[@class='pageNext']")
            # 等待下一頁按鈕出現
            pageNext = wait.until(
                EC.presence_of_element_located((By.XPATH, "//a[@class='pageNext']"))
            )
            # pageNext = wait.until(
            #     EC.element_to_be_clickable((By.XPATH, "//a[@class='pageNext']")) )
            # 使用滾動到可見範圍的方式點擊元素
            # actions = ActionChains(driver)
            # actions.move_to_element(pageNext)
            time.sleep(1)
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", pageNext)
            actions.click(pageNext)
            actions.perform()
            print('---------------切換至下一頁---------------------------')
        except IndexError as e :
            print(f"發生 Error: {e} ")
            continue
    # 結束前一個session
    driver.quit()

------------------------------
爬蟲新北市頁面中
發生 IndexError: list index out of range Retrying...
頁面總數量: 1770
手機號碼 0936-861-025 已存在於df中
手機號碼 0939-563-088 已存在於df中
手機號碼 0939-563-088 連續2次存在於df中，中断迴圈


MoveTargetOutOfBoundsException: Message: move target out of bounds
  (Session info: chrome=119.0.6045.125)
Stacktrace:
	GetHandleVerifier [0x00007FF747D982B2+55298]
	(No symbol) [0x00007FF747D05E02]
	(No symbol) [0x00007FF747BC045D]
	(No symbol) [0x00007FF747C431B9]
	(No symbol) [0x00007FF747C220AA]
	(No symbol) [0x00007FF747C3AAA4]
	(No symbol) [0x00007FF747C21E83]
	(No symbol) [0x00007FF747BF670A]
	(No symbol) [0x00007FF747BF7964]
	GetHandleVerifier [0x00007FF748110AAB+3694587]
	GetHandleVerifier [0x00007FF74816728E+4048862]
	GetHandleVerifier [0x00007FF74815F173+4015811]
	GetHandleVerifier [0x00007FF747E347D6+695590]
	(No symbol) [0x00007FF747D10CE8]
	(No symbol) [0x00007FF747D0CF34]
	(No symbol) [0x00007FF747D0D062]
	(No symbol) [0x00007FF747CFD3A3]
	BaseThreadInitThunk [0x00007FFB34F77344+20]
	RtlUserThreadStart [0x00007FFB364C26B1+33]


#### 資料整理

In [151]:
data591 = pd.read_excel('591房仲手機號碼.xlsx')
data591['accs_num'] = data591['accs_num'].apply(lambda x: x.replace("-", ""))
data591['accs_num'] = data591['accs_num'].str.replace(r'\s', '', regex=True).str[:10] # 將accs_num列中的所有空白去除
data591 = data591.drop_duplicates(subset='accs_num', keep='first') # 刪除重複的號碼
data591 = data591.reset_index(drop=True)
data591.to_csv('591房仲手機號碼.csv', index=False)
print(data591)

        accs_num location
0     0979123933      台中市
1     0909516066      台中市
2     0911750969      台中市
3     0986851077      台中市
4     0986851088      台北市
...          ...      ...
5942  0911056316      桃園市
5943  0938881262      桃園市
5944  0909226988      桃園市
5945  0939889884      桃園市
5946  0930516105      桃園市

[5947 rows x 2 columns]


#### 好房網

In [ ]:
locations_good = {
    '新北市':"https://buy.housefun.com.tw/region/%E6%96%B0%E5%8C%97%E5%B8%82_c/",
    '桃園市':"https://buy.housefun.com.tw/region/%E6%A1%83%E5%9C%92%E5%B8%82_c/",
    '台北市':"https://buy.housefun.com.tw/region/%E5%8F%B0%E5%8C%97%E5%B8%82_c/",
    '新竹縣':"https://buy.housefun.com.tw/region/%E6%96%B0%E7%AB%B9%E7%B8%A3_c/",
    '新竹市':"https://buy.housefun.com.tw/region/%E6%96%B0%E7%AB%B9%E5%B8%82_c/",
    '基隆市':"https://buy.housefun.com.tw/region/%E5%9F%BA%E9%9A%86%E5%B8%82_c/",
    '宜蘭縣':"https://buy.housefun.com.tw/region/%E5%AE%9C%E8%98%AD%E7%B8%A3_c/",
    '台中市':"https://buy.housefun.com.tw/region/%E5%8F%B0%E4%B8%AD%E5%B8%82_c/",
    '苗栗縣':"https://buy.housefun.com.tw/region/%E8%8B%97%E6%A0%97%E7%B8%A3_c/",
    '彰化縣':"https://buy.housefun.com.tw/region/%E5%BD%B0%E5%8C%96%E7%B8%A3_c/",
    '南投縣':"https://buy.housefun.com.tw/region/%E5%8D%97%E6%8A%95%E7%B8%A3_c/",
    '雲林縣':"https://buy.housefun.com.tw/region/%E9%9B%B2%E6%9E%97%E7%B8%A3_c/",
    '高雄市':"https://buy.housefun.com.tw/region/%E9%AB%98%E9%9B%84%E5%B8%82_c/",
    '台南市':"https://buy.housefun.com.tw/region/%E5%8F%B0%E5%8D%97%E5%B8%82_c/",
    '屏東縣':"https://buy.housefun.com.tw/region/%E5%B1%8F%E6%9D%B1%E7%B8%A3_c/",
    '嘉義市':"https://buy.housefun.com.tw/region/%E5%98%89%E7%BE%A9%E5%B8%82_c/",
    '嘉義縣':"https://buy.housefun.com.tw/region/%E5%98%89%E7%BE%A9%E7%B8%A3_c/",
    '花蓮縣':"https://buy.housefun.com.tw/region/%E8%8A%B1%E8%93%AE%E7%B8%A3_c/",
    '台東縣':"https://buy.housefun.com.tw/region/%E5%8F%B0%E6%9D%B1%E7%B8%A3_c/",
    '金門縣':"https://buy.housefun.com.tw/region/%E9%87%91%E9%96%80%E7%B8%A3_c/",
    '澎湖縣':"https://buy.housefun.com.tw/region/%E6%BE%8E%E6%B9%96%E7%B8%A3_c/",
    '連江縣':"https://buy.housefun.com.tw/region/%E9%80%A3%E6%B1%9F%E7%B8%A3_c/"
}


In [ ]:
# 讀取現有的 Excel 檔案
file_path = '好房網房仲手機號碼.xlsx'
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

for loc,web_link in locations_good.items(): #台灣所有地區
    #重新開始新的session
    options = Options()
    #options.add_argument('--headless')  # 選擇無頭瀏覽模式
    driver = webdriver.Chrome(options=options)
    print("------------------------------")
    print("爬蟲{}頁面中".format(loc))
    # 好房網
    older_house = web_link
    wait = WebDriverWait(driver, 10)
    outer_loop_flag = False
    driver.get(older_house)



    # 定義一個變數，表示最大嘗試次數
    max_attempts = 3
    attempts = 0
    consecutive_exist_count = 0

    while attempts < max_attempts:
        try:
            top = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@class='has-top-ad']"))
            )  
            #top = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "has-top-ad")))
            #top = driver.find_element(By.XPATH, "//*[@class='has-top-ad']")
            content = top.find_element(By.XPATH, "//*[@class='l-main-content']")
            m_list = content.find_element(By.XPATH, "//*[@class='m-list']")
            pagination = m_list.find_element(By.XPATH, "//*[@class='m-pagination-bd']")

            final = pagination.find_element(By.XPATH, "//a[text()='最末頁']")
            # step 1 --------取得頁面總數量-----------
            # 獲取 data-value 屬性的值
            last_page_number = final.get_attribute("data-value")
            # 取得頁面總數量
            print(loc,"頁面總數量:",last_page_number)
            # 如果成功找到元素，則離開迴圈
            break
        except IndexError as e :
            print(f"發生 IndexError: {e} Retrying...")
            time.sleep(1)
            attempts += 1



    for page in range(int(last_page_number) - 1): # 換頁執行 直到全部執行完畢
        if outer_loop_flag == True:
            break
        # step 2 --------取得該頁總數量-----------
        # 使用 XPath 定位 <section> 元素 / 使用 XPath 定位具有特定 class 的元素
        try:
            top = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@class='has-top-ad']"))
            )  
            content = top.find_element(By.XPATH, "//*[@class='l-main-content']")
            list_obj = top.find_elements(By.XPATH, "//*[@class='m-list-obj vr-obj ']")    
            # # 記住一開始頁面的窗口號碼
            main_window = driver.window_handles[0]
        except IndexError as e :
            print(f"發生 Error: {e} ")
            continue
        # step 3 --------從第一個開始執行-----------
        for i in range(1,len(list_obj)): #排除第一個 因為第一個推薦的有時長的不一樣
            if outer_loop_flag == True:
                break
            try:
                top = wait.until(
                    EC.presence_of_element_located((By.XPATH, "//*[@class='has-top-ad']"))
                )  
                content = top.find_element(By.XPATH, "//*[@class='l-main-content']")
                list_obj = top.find_elements(By.XPATH, "//*[@class='m-list-obj vr-obj ']")
                # 定位 <a> 元素
                m_list_figure = list_obj[i].find_element(By.CLASS_NAME, "m-list-figure")

                # 定位 <figure> 元素
                figure = m_list_figure.find_element(By.TAG_NAME, "figure")    
                #figure = list_obj[i].find_element(By.XPATH, "//*[@class='m-list-figure']")
                # 使用滾動到可見範圍的方式點擊元素
                actions = ActionChains(driver)
                actions.move_to_element(figure)
                actions.click(figure)
                actions.perform()

                # 記住新窗口的句柄
                new_window_handle = None

                # 等待新窗口或標籤，然後切換到它
                try:
                    new_window_handle = wait.until(lambda driver: len(driver.window_handles) > 1)
                except TimeoutException:
                    print("TimeoutException: No new window found.")
                    continue  # 跳過這一次循環，繼續下一次

                # 更新窗口句柄
                new_window_handle = [handle for handle in driver.window_handles if handle != main_window][0]

                # 切換到新窗口
                driver.switch_to.window(new_window_handle)

                element = wait.until(EC.presence_of_element_located((By.ID , "vue-container")))
                # 使用 XPath 定位 <section> 元素
                #elements2 = element.find_element(By.CLASS_NAME, "img-and-info clearfix")
                elements2 = element.find_element(By.XPATH, "//*[@class='shortcuts-container mb-[50px]']")
                # 取得元素的 outerHTML
                #element2_html = elements2.get_attribute("outerHTML")
                elements3 = elements2.find_element(By.XPATH, "//*[@class='bg-gray w-[300px]']")
                elements4 = elements3.find_element(By.XPATH, "//*[@class='flex flex-col justify-between']")
                elements5 = elements4.find_element(By.XPATH, "//*[@class='font-bold']")

                phone_num = elements5.text
                phone_num = phone_num.strip()
                if not any(cell.value == phone_num for cell in sheet['A']) and phone_num: #hone_num not in df["accs_num"].values:
                    # 寫入新資料
                    last_row = sheet.max_row + 1
                    sheet.cell(row=last_row, column=1, value=phone_num)
                    sheet.cell(row=last_row, column=2, value=loc)
                    # 儲存檔案
                    workbook.save(file_path)
                    print('手機號碼 {} 寫入df'.format(phone_num))
                    # 重置計數器
                    consecutive_exist_count = 0

                    # 關閉原始窗口
                    driver.close()

                    # 切換回主窗口
                    driver.switch_to.window(main_window)
                else:
                    print('手機號碼 {} 已存在於df中'.format(phone_num))
                    consecutive_exist_count += 1

                    # 關閉原始窗口
                    driver.close()

                    # 切換回主窗口
                    driver.switch_to.window(main_window)

                    # 如果連續存在35次，中斷迴圈
                    if consecutive_exist_count >= 61:
                        print('手機號碼 {} 連續61次存在於df中，中断迴圈'.format(phone_num))
                        outer_loop_flag = True
                        break
            except Exception as e:
                # 捕獲例外並處理
                print(f"An error occurred: {e}")
                continue
        # step 4 --------切換下一頁-----------
        # 使用 XPath 定位 <section> 元素 / 使用 XPath 定位具有特定 class 的元素
        try:
            top = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@class='has-top-ad']"))
            )  
            content = top.find_element(By.XPATH, "//*[@class='l-main-content']")
            m_list = content.find_element(By.XPATH, "//*[@class='m-list-bd']")
            pagination = m_list.find_element(By.XPATH, "//*[@class='m-pagination-bd']")
            pageNext = pagination.find_element(By.XPATH, "//*[@class='has-arrow']")
            actions = ActionChains(driver)
            actions.move_to_element(next)
            # driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", pageNext)
            actions.click(pageNext)
            actions.perform()
            print('---------------切換至下一頁---------------------------')
        except IndexError as e :
            print(f"發生 Error: {e} ")
            continue
    # 結束前一個session
    driver.quit()

------------------------------
爬蟲新北市頁面中
新北市 頁面總數量: 527
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼  已存在於df中
手機號碼  已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼  已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼  已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169966 已存在於df中
---------------切換至下一頁---------------------------
手機號碼 02-33169966 已存在於df中
手機號碼  已存在於df中
手機號碼  已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169966 已存在於df中
手機號碼 02-33169966 已存在於df中
---------------切換至下一頁---------------------------
手機號碼 02-33169977 已存在於df中
手機號碼 02-33169977 連續35次存在於df中，中断迴圈
----

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF747D982B2+55298]
	(No symbol) [0x00007FF747D05E02]
	(No symbol) [0x00007FF747BC05AB]
	(No symbol) [0x00007FF747BA0038]
	(No symbol) [0x00007FF747C26BC7]
	(No symbol) [0x00007FF747C3A15F]
	(No symbol) [0x00007FF747C21E83]
	(No symbol) [0x00007FF747BF670A]
	(No symbol) [0x00007FF747BF7964]
	GetHandleVerifier [0x00007FF748110AAB+3694587]
	GetHandleVerifier [0x00007FF74816728E+4048862]
	GetHandleVerifier [0x00007FF74815F173+4015811]
	GetHandleVerifier [0x00007FF747E347D6+695590]
	(No symbol) [0x00007FF747D10CE8]
	(No symbol) [0x00007FF747D0CF34]
	(No symbol) [0x00007FF747D0D062]
	(No symbol) [0x00007FF747CFD3A3]
	BaseThreadInitThunk [0x00007FFB34F77344+20]
	RtlUserThreadStart [0x00007FFB364C26B1+33]
